# Optimization for Data Science 2024 Homework 1

Students:

Alberto Calabrese Nº:2103405

Greta d'Amore Grelli Nº:2122424

Eleonora Mesaglio Nº:2103402

Marlon Helbing Nº:2106578

## 1. Generating the dataset

In [1]:
import numpy as np

# Set a seed for deterministic outputs
SEED = 42
np.random.seed(seed = SEED)

### A - MATRIX

In [2]:
NUM_SAMPLES = 1000
NUM_FEATURES = 1000

# Generate a 1000x1000 matrix with random samples from a standard normal distribution
# This is our data matrix, which contains 1000 samples (rows) with 1000 features each (columns)
# A MATRIX
data_matrix = np.random.normal(0, 1, size = (NUM_SAMPLES, NUM_FEATURES)) # He refers to it as A 
# Now 'data_matrix' contains random values drawn from N(0,1)

### X - MATRIX

In [3]:
NUM_LABELS = 50

# This is our weight matrix that we initialize like this ; these weights we want to learn
# it has 1000 features (rows) with 50 labels each (columns)
# X MATRIX
weight_matrix = np.random.normal(0, 1, size = (NUM_FEATURES, NUM_LABELS)) # He refers to it as X 

### E - MATRIX

In [4]:
NUM_EXAMPLES = 1000

# This matrix is used to help generating our supervised gold labels 
# It is of size 1000 training examples (rows) and their labels (columns)
generative_matrix = np.random.normal(0, 1, size = (NUM_EXAMPLES, NUM_LABELS)) # He refers to it as E 

### AX + E

In [5]:
# Create a vector with numbers from 1 to 50
label_vector = np.arange(1, 51)

# Print the vector
#print(label_vector)

In [6]:
# Now he wants us to calculate AX+E to generate labels for the 1000 training examples (such that we have a supervised learning set) +

# Calculate the matrix product AX
AX = np.matmul(data_matrix, weight_matrix)  # or simply: AX = A @ X

# Add E to AX element-wise
result_matrix = AX + generative_matrix

print(result_matrix.shape)

(1000, 50)


## 2. MAX INDEX AS CLASS LABEL

In [7]:
# We find our labels by considering the max index in the row as the class label

# Find the column indices of maximum values for each row
max_indices = np.argmax(result_matrix, axis=1)

print(max_indices)

print(result_matrix[2,:])
print(max_indices.shape)

# 'max_indices' now contains the column indices of maximum values for each row

[ 7 43 44  7 49 21 10 15 41 43 33 45  8 31 35 29 28 46 36 36  4 23 14  0
  8 29 20 31  8  7 27 47 12 11 33  3  5 16  7 25 10 46 27 41 28  0  1  2
 20 48 39 49 36 10 32  4 22 15 11 19 20 30 17 38 49 15 15 12 11  4  5 35
  5  4 36  5 34 47 15  0 34  2 35 38 41 21 41 14 29 24  7  0 30  0  9 29
 24 12 21 45 49 23  7 26 21  5 36 43 42 30 25 26 39  6 44 21 26 37  0 36
 47 22  2 46  5  4 10 10 17  3 43  1 20  9 28 11  9 48 23 35 26 11 27 36
 28 27 42 35 35 16 34 42 12  2  7 44 41 40 46 23 33  9  0 42 26 43 29 15
 13  7  4  4 33 10  7 32 16 32 30 15 32 48 21  7 12 22 32 19 17 13 26 38
 38  3 31 35 19 49 20 39 34 36  4 21 24 21 47 26 39 43 33 28 45 21 13 22
 23 15 37 23 30  9 38 44 10  1 27 37 10 16 24  3 29 21  5 25 40 36 26 22
 37 12 18 10 10 13  4 39 22 19 33 12 16 44 11 22 21 12  4 45 43 28  9 27
 48 40 41 13 15 34 32 36 41 19 43 23  6  9 41 40 18 23 28 41 26 30 15 28
 27 36 10 34 16 14  8 49  9 47 21 49 28  9 34 17 45 25 48  6 36 25 38 11
 26 31 48 49 27 21 33  9  1  7 43 37 15 15 21 10 36

## 3. Gradient Descent

In [8]:
# Define the negative log-likelihood function
def cost_func(data_matrix, weight_matrix, labels):
    scores = np.dot(data_matrix, weight_matrix)
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    corect_logprobs = -np.log(probs[range(NUM_EXAMPLES), labels])
    data_loss = np.sum(corect_logprobs)
    return data_loss


# Define the function to compute the gradient of the negative log-likelihood function
def gradient(data_matrix, weight_matrix, labels):
    scores = np.dot(data_matrix, weight_matrix)
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    probs[range(NUM_EXAMPLES), labels] -= 1
    dW = np.dot(data_matrix.T, probs)
    return dW

In [9]:
# Define the learning rate and the number of iterations
learning_rate = 0.01
num_iterations = 1000

# Perform gradient descent
for i in range(num_iterations):
    grad = gradient(data_matrix, weight_matrix, max_indices)
    weight_matrix -= learning_rate * grad

    if i % 100 == 0:
        print("iteration %d: loss %f" % (i, cost_func(data_matrix, weight_matrix, max_indices)))

iteration 0: loss 4.146766
iteration 100: loss 0.207946
iteration 200: loss 0.116930
iteration 300: loss 0.082825
iteration 400: loss 0.064673
iteration 500: loss 0.053311
iteration 600: loss 0.045492
iteration 700: loss 0.039762
iteration 800: loss 0.035373
iteration 900: loss 0.031897


In [10]:
NUM_LABELED = 500
Y_0 = np.random.rand(NUM_SAMPLES, NUM_LABELS) # define an appropriate starting point
assert Y_0.shape == (NUM_SAMPLES, NUM_LABELS)

EPSILON = 1e-6 # define small epsilon for stopping criterion
MAX_ITER = 2000 # and/or a maximum number of iterations (or even a maximum time)

ALPHA = 0.01 # define a fixed stepsize

In [11]:
import time

Y_iterates = [Y_0]
times = [0]
start = time.time()

grad = gradient(data_matrix, weight_matrix, max_indices)
while len(Y_iterates) < MAX_ITER and np.linalg.norm(grad) > EPSILON: # TO DO: write the condition for the while loop
    new_y = Y_iterates[-1] - ALPHA * grad # write the update
    Y_iterates.append(new_y)
    times.append(time.time() - start)
    # Check the stopping criterion
    grad = gradient(data_matrix, weight_matrix, max_indices)


In [12]:
import matplotlib.pyplot as plt

plt.plot(times, [cost_func(data_matrix, weight_matrix, y.astype(int)) for y in Y_iterates])
plt.xlabel('CPU time (seconds)')
plt.ylabel('Objective function')
plt.yscale("log")
plt.show()

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (1000,) (1000,50) 